<script src="https://use.fontawesome.com/2e576da815.js"></script>

<div style="text-align: left;"><img src="https://www.juliabox.org/assets/img/juliacloudlogo.png" style="margin: 0px 0px 0px 0px; padding-right: 20px;width: 80px; float: left;" title="" alt="" /></div>
<img src="http://dmkpress.com/images/cms/thumbs/a5b0aeaa3fa7d6e58d75710c18673bd7ec6d5f6d/978-5-97060-370-3_270_369__100.jpg" style="margin: 0px 0px 5px 20px; width: 100px; float: right;" title="" alt="" />
Всестороннее введение в новый язык программирования для научно-технических вычислений [Julia](http://julialang.org/) в книге Малколма Шеррингтона, Packt Publishing, июль 2015.

<h1>Осваиваем язык Julia</h1><br />

Совершенствование мастерства в области аналитики и программирования при помощи Julia в целях решения задач комплексной обработки данных
<div style="text-align: left;font-size:8pt;padding-top:10px;">Программный код Julia (v0.4.5) протестирован в Windows 8.1/10 и Linux/Lubuntu 16.4</div>
<div style="text-align: left;"><h1>Глава 4. Функциональная совместимость</h1></div>

# 3. Метапрограммирование
## Символьные имена

In [7]:
ex = :(a+b*c) 

:(a + b * c)

In [8]:
typeof(ex)

Expr

In [9]:
a = 1.0; b = 2.5; c = 23 + 4im;
eval(ex) 

58.5 + 10.0im

In [10]:
fieldnames(Expr) 

3-element Array{Symbol,1}:
 :head
 :args
 :typ 

In [11]:
ex.args   

3-element Array{Any,1}:
 :+      
 :a      
 :(b * c)

In [12]:
ex.args[3].args  

3-element Array{Any,1}:
 :*
 :b
 :c

In [13]:
ex.head 

:call

In [14]:
ex.typ

Any

## Макрокоманды

<i class="fa fa-linux" aria-hidden="true"></i>
<i class="fa fa-apple" aria-hidden="true"></i>

In [34]:
systime() = ccall( (:time, "libc"), Int32, ());
macro uxtime(ex)
  quote
    t0 = systime()
    $(esc(ex))
    t1 = systime()
    println("Время обработки составило $(t1 - t0) сек.")
  end
end 

In [ ]:
include(homedir()*"/julia_projects/queens.jl")

@uxtime qsolve(8)   # прим. 53 сек. 

<i class="fa fa-windows" aria-hidden="true"></i>
<i class="fa fa-linux" aria-hidden="true"></i>
<i class="fa fa-apple" aria-hidden="true"></i>

In [6]:
function hangabout(n)
  s = 0.0
  for i = 1:n
    s += i/(i+1)^2
  end
  s
end

hangabout (generic function with 1 method)

In [7]:
hangabout(1_000_000_000) 

19.655547437502335

In [15]:
@elapsed hangabout(1_000_000_000)

7.164108306

In [16]:
macroexpand(quote @elapsed hangabout(1_000_000_000) end)

quote  # In[16], line 1:
    begin  # util.jl, line 178:
        local #62#t0 = Base.time_ns() # util.jl, line 179:
        local #63#val = hangabout(1000000000) # util.jl, line 180:
        Base./(Base.-(Base.time_ns(),#62#t0),1.0e9)
    end
end

In [17]:
macro benchmark(f)
  quote
    $(esc(f))
    mean([@elapsed $(esc(f)) for i = 1:10])
  end
end

In [18]:
macroexpand(quote @benchmark hangabout(1_000_000_000) end)

quote  # In[18], line 1:
    begin  # In[17], line 3:
        hangabout(1000000000) # In[17], line 4:
        mean([begin  # util.jl, line 178:
                local #65#t0 = Base.time_ns() # util.jl, line 179:
                local #66#val = hangabout(1000000000) # util.jl, line 180:
                Base./(Base.-(Base.time_ns(),#65#t0),1.0e9)
            end for #64#i = 1:10])
    end
end

## Тестирование

In [19]:
using Base.Test

x = 1;
@test x == 1
@test x == 2  # => ErrorException("test failed: :((x==2))")

LoadError: LoadError: test failed: 1 == 2
 in expression: x == 2
while loading In[19], in expression starting on line 5

In [21]:
a = rand(10);

@test_throws BoundsError a[11] = 0.1
@test_throws DomainError a[11] = 0.1

LoadError: LoadError: test failed: BoundsError([0.3284644758041464,0.7137241560117829,0.4093934847398273,0.9864315978029479,0.8909286447186917,0.011079639569080424,0.7836313605916725,0.6389251344519136,0.9335210321123686,0.10798471482581484],(11,)) was thrown instead of DomainError
 in expression: a[11] = 0.1
while loading In[21], in expression starting on line 4

In [22]:
@test_approx_eq 0.0 sin(pi)

LoadError: LoadError: assertion failed: |0.0 - sin(pi)| <= 2.465190328815662e-28
  0.0 = 0.0
  sin(pi) = 1.2246467991473532e-16
  difference = 1.2246467991473532e-16 > 2.465190328815662e-28
while loading In[22], in expression starting on line 1

In [23]:
@test_approx_eq_eps 0.0 sin(pi) 1.0e-10
@test_approx_eq_eps 0.0 sin(pi) 1.0e-20

LoadError: LoadError: assertion failed: |0.0 - sin(pi)| <= 1.0e-20
  0.0 = 0.0
  sin(pi) = 1.2246467991473532e-16
  difference = 1.2246467991473532e-16 > 1.0e-20
while loading In[23], in expression starting on line 2

### Обработка ошибок

In [24]:
using Base.Test

my_handler(r::Test.Success) = println("Согласен с $(r.expr)");
my_handler(r::Test.Failure) = error("Ошибка из моего обработчика: $(r.expr)")
my(r::Test.Error) = rethrow(r)

my (generic function with 1 method)

In [25]:
fieldnames(Test.Error) 

3-element Array{Symbol,1}:
 :expr     
 :err      
 :backtrace

In [26]:
my_handler(r::Test.Failure) = println("$(r.expr): да, как и задумано!");
my_handler(r::Test.Success) = println("$(r.expr): вполне устраивает");
my_handler(r::Test.Error) = rethrow(r);

In [28]:
Test.with_handler(my_handler) do
  x = 1;
  @test x == 1
  @test x == 2
  @test x / 0
end

x == 1
x == 2  
x / 0

x == 1: вполне устраивает
x == 2: да, как и задумано!


LoadError: LoadError: test error in expression: x / 0
TypeError: non-boolean (Float64) used in boolean context
 in do_test at test.jl:51
while loading In[28], in expression starting on line 1

## Макрокоманда enum

In [1]:
@enum STATUS INFO WARNING ERROR FATAL 

In [2]:
INFO         

INFO::STATUS

In [3]:
STATUS(0)   

INFO::STATUS

In [4]:
STATUS(1)    

In [5]:
typeof(INFO) 

STATUS

Чтобы разобраться, как работает встроенная макрокоманда @enum, обратимся к ее определению.

In [38]:
macro enum2(T,syms...)
  blk = quote
    immutable $(esc(T))
      n::Int32
      $(esc(T))(n::Integer) = new(n)
    end
  
    Base.show(io::IO, x::$(esc(T))) = print(io, $syms[x.n+1])
    Base.show(io::IO, x::Type{$(esc(T))}) = print(io, $(string("enum ", T, ' ', '(', join(syms, ", "), ')')) )
  end

  for (i,sym) in enumerate(syms)
    push!(blk.args, :(const $(esc(sym)) = $(esc(T))($(i-1))))
  end

  push!(blk.args, :nothing)
  blk.head = :toplevel

  return blk
end

In [39]:
# как вариант, можно включить тип, указав его расположение:
# cd(joinpath(homedir(),"julia_projects","code"))
# include("enum2.jl")

@enum2 MSGSTAT INFO2 WARN2 ERROR2 FATAL2

typealias MsgStatus typeof(INFO2)  

MSGSTAT

In [40]:
type LogMessage2
  stamped::DateTime
  msgstat::MsgStatus
  message::AbstractString
end

import Base.show
show(m::LogMessage2) = print("$(m.stamped): $(m.msgstat) >> $(m.message)")
msg = LogMessage2(now(), INFO2, "Буду очень обрадован!")

show(msg)   

2016-05-21T00:44:12: INFO2 >> Буду очень обрадован!

In [10]:
@enum ФРУКТ яблоко=1 апельсин=2 киви=3

f(x::ФРУКТ) = "Я фрукт со значением $(Int(x))"

f(яблоко)

"Я фрукт со значением 1"